In [0]:
WITH RandomProduct AS (
  SELECT
      ROW_NUMBER() OVER (ORDER BY RAND()) + 100 AS row,
      *
  FROM shm.marketing.products_sample
  LIMIT 100
)
SELECT 
  C.campaign_id, C.template, C.cost, 
  C.start_date, C.end_date, C.mailing_list,
  P.title, P.product_id, 
  json_tuple(
    AI_QUERY(
      'databricks-meta-llama-3-3-70b-instruct',
      CONCAT('Using the following product title, description, and feature summary, make a witty campaign name, campaign description, and email marketing subject line. Return them in json format. \n', title, product_description, features_summary),
      responseFormat => '{
        "type": "json_schema",
        "json_schema": {
          "name": "generated_campaign",
          "schema": {
            "type": "object",
            "properties": {
              "campaign_name": {"type": "string"},
              "campaign_description": {"type": "string"},
              "subject_line": {"type": "string"}
            }
          }
        }
      }'
    ), 
    'campaign_name', 'campaign_description', 'subject_line') 
    AS (campaign_name, campaign_description, subject_line)
FROM RandomProduct P
JOIN shm.marketing.campaigns C
ON P.row = C.campaign_id